<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Exploratory" data-toc-modified-id="Exploratory-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Exploratory</a></span><ul class="toc-item"><li><span><a href="#Content-of-the-files" data-toc-modified-id="Content-of-the-files-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Content of the files</a></span></li><li><span><a href="#Books" data-toc-modified-id="Books-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Books</a></span><ul class="toc-item"><li><span><a href="#Metadata" data-toc-modified-id="Metadata-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Metadata</a></span></li><li><span><a href="#Reviews" data-toc-modified-id="Reviews-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Reviews</a></span></li><li><span><a href="#5-Core" data-toc-modified-id="5-Core-1.2.3"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>5-Core</a></span></li></ul></li><li><span><a href="#Movies-and-TV" data-toc-modified-id="Movies-and-TV-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Movies and TV</a></span><ul class="toc-item"><li><span><a href="#Metadata" data-toc-modified-id="Metadata-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Metadata</a></span></li><li><span><a href="#Reviews" data-toc-modified-id="Reviews-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Reviews</a></span></li></ul></li><li><span><a href="#Electronics" data-toc-modified-id="Electronics-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Electronics</a></span><ul class="toc-item"><li><span><a href="#Metadata" data-toc-modified-id="Metadata-1.4.1"><span class="toc-item-num">1.4.1&nbsp;&nbsp;</span>Metadata</a></span></li><li><span><a href="#Review" data-toc-modified-id="Review-1.4.2"><span class="toc-item-num">1.4.2&nbsp;&nbsp;</span>Review</a></span></li></ul></li></ul></li><li><span><a href="#Detecting-similar-products" data-toc-modified-id="Detecting-similar-products-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Detecting similar products</a></span><ul class="toc-item"><li><span><a href="#Books" data-toc-modified-id="Books-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Books</a></span><ul class="toc-item"><li><span><a href="#Defining-the-dataset-to-work-with" data-toc-modified-id="Defining-the-dataset-to-work-with-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Defining the dataset to work with</a></span></li><li><span><a href="#Locality-sensitive-hashing" data-toc-modified-id="Locality-sensitive-hashing-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Locality sensitive hashing</a></span><ul class="toc-item"><li><span><a href="#The-minhash" data-toc-modified-id="The-minhash-2.1.2.1"><span class="toc-item-num">2.1.2.1&nbsp;&nbsp;</span>The <code>minhash</code></a></span></li><li><span><a href="#The-principle-of-LSH" data-toc-modified-id="The-principle-of-LSH-2.1.2.2"><span class="toc-item-num">2.1.2.2&nbsp;&nbsp;</span>The principle of LSH</a></span></li><li><span><a href="#Finding-candidate-duplicates" data-toc-modified-id="Finding-candidate-duplicates-2.1.2.3"><span class="toc-item-num">2.1.2.3&nbsp;&nbsp;</span>Finding candidate duplicates</a></span></li></ul></li><li><span><a href="#Refining-our-candidates" data-toc-modified-id="Refining-our-candidates-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Refining our candidates</a></span><ul class="toc-item"><li><span><a href="#Using-Amazon-API-to-obtain-more-data-about-our-products" data-toc-modified-id="Using-Amazon-API-to-obtain-more-data-about-our-products-2.1.3.1"><span class="toc-item-num">2.1.3.1&nbsp;&nbsp;</span>Using Amazon API to obtain more data about our products</a></span></li><li><span><a href="#Refining-the-candidate-similars-based-on-authors" data-toc-modified-id="Refining-the-candidate-similars-based-on-authors-2.1.3.2"><span class="toc-item-num">2.1.3.2&nbsp;&nbsp;</span>Refining the candidate similars based on authors</a></span></li><li><span><a href="#Refining-the-candidate-similars-further-using-the-titles" data-toc-modified-id="Refining-the-candidate-similars-further-using-the-titles-2.1.3.3"><span class="toc-item-num">2.1.3.3&nbsp;&nbsp;</span>Refining the candidate similars further using the titles</a></span></li></ul></li></ul></li></ul></li><li><span><a href="#Testing" data-toc-modified-id="Testing-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Testing</a></span></li></ul></div>

In [1]:
%matplotlib inline
# To reload external scripts automatically
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import sys

# Importing external files
sys.path.append('scripts/')
from data_import import *
from similarities import *
from utils import *
from amazon_api_interaction import *

pd.options.mode.chained_assignment = None  # default='warn', Mutes warnings when copying a slice from a DataFrame.

# Exploratory
This notebook contain the first study of the data to identify what is needed and what we do not need. It will only work with subsets of datasets as they will not fit into memory.

In [2]:
DATA_FOLDER = "../../Project-Data/"
META_FOLDER = DATA_FOLDER + "meta/"
REVIEWS_FOLDER = DATA_FOLDER + "reviews/"
CORE_FOLDER = DATA_FOLDER + "5_core/"
DUMP_FOLDER = DATA_FOLDER + "dump/"
CATEGORIES = ['Books','Movies_and_Tv','Electronics']
MAXCOUNT = -1

## Content of the files

The different columns of the **metadata** files are : 
* ```asin``` : the unique identifier of the object
* ```brand```
* ```categories``` : the categories of the object
* ```description``` : the description of the object
* ```imUrl```  : the link toward the images related to the object
* ```price```
* ```related``` : a list of objects that are related to this object
* ```salesRank``` 
* ```title```

The different columns of the **reviews** and **5-core** files are :
* ```asin``` : the unique identifier of the object
* ```helpful``` : a list of 2 integers [x,y], the helpfulness score is x/y votes
* ```overall``` : the rating of the object
* ```reviewText```
* ```reviewTime```
* ```reviewerID```
* ```reviewerName```
* ```summary ``` : the title of the review
* ```unixReviewTime``` : in Unix format

Therefore we keep only the column that are of interest for our task

In [3]:
meta_interesting_cols = ['asin', 'title', 'salesRank', 'description','imUrl']
review_interesting_cols = ['asin', 'overall', 'unixReviewTime']

## Books

In [4]:
meta_books_path, review_books_path, core_book_path = get_paths(0, DATA_FOLDER, META_FOLDER,CORE_FOLDER,
                                               REVIEWS_FOLDER, CATEGORIES)

Paths : 
	 meta = ../../Project-Data/meta/meta_Books.json
	 review = ../../Project-Data/reviews/reviews_Books.json
	 core_path = ../../Project-Data/5_core/Books.json


### Metadata

In [5]:
meta_books = import_interesting_cols(meta_books_path,DUMP_FOLDER,True,meta_interesting_cols,max_count=MAXCOUNT)
meta_books.head(1)

Retrieving from : ../../Project-Data/dump/meta_Books_asin_title_salesRank_description_imUrl_ALL
It took 00:00:05.529 to import the data.


,asin,description,imUrl,"salesRank_Arts,_Crafts_&_Sewing",salesRank_Books,salesRank_Cell_Phones_&_Accessories,salesRank_Clothing,salesRank_Electronics,salesRank_Health_&_Personal_Care,salesRank_Home_&_Kitchen,...,salesRank_Jewelry,salesRank_Kitchen_&_Dining,salesRank_Movies_&_TV,salesRank_Music,salesRank_Musical_Instruments,salesRank_Office_Products,salesRank_Shoes,salesRank_Sports_&_Outdoors,salesRank_Toys_&_Games,title
0,0001048791,NaN,http://ecx.images-amazon.com/images/I/51MKP0T4...,NaN,6334800.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"The Crucible: Performed by Stuart Pankin, Jero..."


### Reviews

In [6]:
review_books = import_interesting_cols(
    review_books_path, DUMP_FOLDER,False, review_interesting_cols, max_count=MAXCOUNT)
review_books.head()

Retrieving from : ../../Project-Data/dump/reviews_Books_asin_overall_unixReviewTime_ALL
It took 00:00:02.994 to import the data.


,asin,overall,unixReviewTime
0,0000000116,4.0,2002-04-27
1,0000000116,1.0,2014-03-24
2,0000000868,4.0,2002-09-11
3,0000013714,4.0,2013-10-31
4,0000013714,5.0,2013-07-27


### 5-Core

In [7]:
core_books = import_interesting_cols(
    core_book_path,
    DUMP_FOLDER,
    False,
    review_interesting_cols,
    max_count=MAXCOUNT)
core_books.head()

Retrieving from : ../../Project-Data/dump/Books_asin_overall_unixReviewTime_ALL
It took 00:00:00.834 to import the data.


,asin,overall,unixReviewTime
0,000100039X,5.0,2012-12-16
1,000100039X,5.0,2003-12-11
2,000100039X,5.0,2014-01-18
3,000100039X,5.0,2011-09-27
4,000100039X,5.0,2002-10-07


## Movies and TV

In [8]:
meta_movie_path, review_movie_path, core_movie_path = get_paths(
    1, DATA_FOLDER, META_FOLDER, CORE_FOLDER, REVIEWS_FOLDER, CATEGORIES)

Paths : 
	 meta = ../../Project-Data/meta/meta_Movies_and_Tv.json
	 review = ../../Project-Data/reviews/reviews_Movies_and_Tv.json
	 core_path = ../../Project-Data/5_core/Movies_and_Tv.json


### Metadata

In [9]:
meta_movie = import_interesting_cols(
    meta_movie_path,DUMP_FOLDER, True, meta_interesting_cols, max_count=MAXCOUNT, dropna=False)
meta_movie.head()

Retrieving from : ../../Project-Data/dump/meta_Movies_and_Tv_asin_title_salesRank_description_imUrl_ALL
It took 00:00:00.748 to import the data.


,asin,description,imUrl,"salesRank_Arts,_Crafts_&_Sewing",salesRank_Books,salesRank_Electronics,salesRank_Health_&_Personal_Care,salesRank_Home_&_Kitchen,salesRank_Industrial_&_Scientific,salesRank_Movies_&_TV,salesRank_Music,salesRank_Pet_Supplies,salesRank_Software,salesRank_Sports_&_Outdoors,salesRank_Toys_&_Games,salesRank_Video_Games,title
0,0000143561,"3Pack DVD set - Italian Classics, Parties and ...",http://g-ecx.images-amazon.com/images/G/01/x-s...,NaN,NaN,NaN,NaN,NaN,NaN,376041.0,NaN,NaN,NaN,NaN,NaN,NaN,"Everyday Italian (with Giada de Laurentiis), V..."
1,0000589012,NaN,http://ecx.images-amazon.com/images/I/519%2B1k...,NaN,NaN,NaN,NaN,NaN,NaN,1084845.0,NaN,NaN,NaN,NaN,NaN,NaN,Why Don't They Just Quit? DVD Roundtable Discu...
2,0000695009,NaN,http://g-ecx.images-amazon.com/images/G/01/x-s...,NaN,NaN,NaN,NaN,NaN,NaN,1022732.0,NaN,NaN,NaN,NaN,NaN,NaN,Understanding Seizures and Epilepsy DVD
3,000107461X,NaN,http://ecx.images-amazon.com/images/I/41WY47gL...,NaN,NaN,NaN,NaN,NaN,NaN,954116.0,NaN,NaN,NaN,NaN,NaN,NaN,Live in Houston [VHS]
4,0000143529,Disc 1: Flour Power (Scones; Shortcakes; South...,http://ecx.images-amazon.com/images/I/51QY79CD...,NaN,NaN,NaN,NaN,NaN,NaN,463562.0,NaN,NaN,NaN,NaN,NaN,NaN,My Fair Pastry (Good Eats Vol. 9)


### Reviews

In [10]:
review_movie = import_interesting_cols(
    review_movie_path,
    DUMP_FOLDER,
    False,
    review_interesting_cols,
    max_count=100,
    dropna=False)
review_movie.head()

Retrieving from : ../../Project-Data/dump/reviews_Movies_and_Tv_asin_overall_unixReviewTime_100
It took 00:00:00.004 to import the data.


,asin,overall,unixReviewTime
0,0000143502,5.0,2013-01-17
1,0000143529,5.0,2013-10-02
2,0000143561,2.0,2008-07-17
3,0000143588,5.0,2009-03-13
4,0000143588,5.0,2009-01-18


## Electronics

In [11]:
meta_electronic_path, review_electronic_path, core_electronic_path = get_paths(
    2, DATA_FOLDER,META_FOLDER,CORE_FOLDER, REVIEWS_FOLDER, CATEGORIES)

Paths : 
	 meta = ../../Project-Data/meta/meta_Electronics.json
	 review = ../../Project-Data/reviews/reviews_Electronics.json
	 core_path = ../../Project-Data/5_core/Electronics.json


### Metadata

In [12]:
meta_electronic = import_interesting_cols(
    meta_electronic_path,
    DUMP_FOLDER,
    True,
    meta_interesting_cols,
    max_count=MAXCOUNT,
    dropna=False)
meta_electronic.head(2)

Retrieving from : ../../Project-Data/dump/meta_Electronics_asin_title_salesRank_description_imUrl_ALL
It took 00:00:02.134 to import the data.


,asin,description,imUrl,"salesRank_Arts,_Crafts_&_Sewing",salesRank_Automotive,salesRank_Baby,salesRank_Beauty,salesRank_Camera_&_Photo,salesRank_Cell_Phones_&_Accessories,salesRank_Clothing,...,salesRank_Office_Products,"salesRank_Patio,_Lawn_&_Garden",salesRank_Pet_Supplies,salesRank_Shoes,salesRank_Software,salesRank_Sports_&_Outdoors,salesRank_Toys_&_Games,salesRank_Video_Games,salesRank_Watches,title
0,0132793040,The Kelby Training DVD Mastering Blend Modes i...,http://ecx.images-amazon.com/images/I/31JIPhp%...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kelby Training DVD: Mastering Blend Modes in A...
1,0321732944,NaN,http://ecx.images-amazon.com/images/I/31uogm6Y...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kelby Training DVD: Adobe Photoshop CS5 Crash ...


### Review

In [13]:
review_electronic = import_interesting_cols(
    review_electronic_path,
    DUMP_FOLDER,
    False,
    review_interesting_cols,
    max_count=MAXCOUNT,
    dropna=False)
review_electronic.head()

Retrieving from : ../../Project-Data/dump/reviews_Electronics_asin_overall_unixReviewTime_ALL
It took 00:00:00.788 to import the data.


,asin,overall,unixReviewTime
0,0132793040,5.0,2013-04-13
1,0321732944,5.0,2012-07-01
2,0439886341,1.0,2013-04-29
3,0439886341,3.0,2013-07-22
4,0439886341,1.0,2012-04-18


# Detecting similar products
In this section we try to obtain the products that are similar among a given category and to develop a systematic way to detect those similar products.

## Books

### Defining the dataset to work with

Some books are sold in different formats : hard cover, pocket, electronic, etc. Therefore in order to compare them we will focus on the title and description that are the only two attributes that should be highly similar between the two different yet similar books.

In [14]:
core_asin = list(core_books['asin'].unique())

# We delete all the rows where either the title or description is null and keep only core books
core_meta_df = meta_books[meta_books['title'].notnull()]
core_meta_df = core_meta_df[core_meta_df['description'].notnull()]
core_meta_df = core_meta_df[core_meta_df['imUrl'].notnull()]
core_meta_df = core_meta_df[core_meta_df['asin'].isin(core_asin)]
book_desc_titles = core_meta_df[['asin','title','description','imUrl']]
book_desc_titles = book_desc_titles.set_index(['asin'])
book_desc_titles.head()

,title,description,imUrl
asin,,,
0001055178,Master Georgie,Beryl Bainbridge seems drawn to disaster. Firs...,http://ecx.images-amazon.com/images/I/51ZSC6TK...
000171287X,The Berenstains' B Book (Bright & Early Books),"By Stan Berenstain and Jan Berenstain, Illustr...",http://ecx.images-amazon.com/images/I/31hjUxZu...
000100039X,The Prophet,"In a distant, timeless place, a mysterious pro...",http://ecx.images-amazon.com/images/I/81ZKLPiv...
0001473905,Rightly Dividing the Word,--This text refers to thePaperbackedition.,http://ecx.images-amazon.com/images/I/61KPC59B...
0001714538,The Berenstain Bears on the Moon (Bright and E...,PreSchool-Grade 2 A delightful tale told in rh...,http://ecx.images-amazon.com/images/I/A1zj84uC...


In [15]:
book_desc_titles.index.is_unique

True

We first try to experiment with tf-idf weights to find common titles among books. Due to the large size of the database, this task cannot be executed on all the books, we decided to focus as suggested by the dataset on the 5-core dataset. The 5 core dataset contains only products and reviews such that reviewers and products have at least 5-reviews. This will allow us in a first analysis to reduce the number of products that we consider. Since we will need to have at least 5 reviews anyway to look at the **Herding effect**.

Let's explore a bit what the dataset contains. we have much informations (and in an heterogeneous manner) in the description. Therefore we might want to observe the name of the auhtor and the title when they are available.

### Locality sensitive hashing
Based on this [article](http://dataconomy.com/2017/06/locality-sensitive-hashing-pydata/), and following the theory that can be read at the chapter 3.4 of [Mining of massive datasets by Leskovec, Rajaraman and Milliway](http://infolab.stanford.edu/~ullman/mmds/book.pdf) we will try to find similar books.

#### The ```minhash```

We need to detect near similar books in this large datasets. To do so we would need to compare each of the attribute of a given book with all the others which is extremely computationaly expensive. There we will use LSH. Right now the title and description of the book have variable length depending on the books. To perform LSH we would like to have a fixed length representation of the document without modifying the semantics of document similarity.
* first we introduce the principle of shingles. A shingle of 5-gram (hence we discard the last shingle as it only consists in the $<5$ last characters of the document) e.g. is a set of all possible 5-grams in the string. 

In [16]:
document = "Lorem Ipsum dolor sit amet"
shingles = get_shingles(document)

In [17]:
other_document = 'Lorem Ipsum dolor sit amet with some extra garbage'
other_shingles = get_shingles(other_document)
jaccard_dist(shingles,other_shingles)

0.4666666666666667

Each document will still have a different number of shingles depending on its length. Therefore we wish to represent it using a *fixed length representation*. We will present now a function ```minhash``` that has a collision probability that is exactly the jaccard similarity (based on this [document](http://infolab.stanford.edu/~ullman/mmds/ch3.pdf) and following this [repo](https://github.com/mattilyra/LSH/blob/master/examples/Introduction.ipynb) . We explain this now :
1. Suppose that the following dataframe is the boolean variable corresponding to whether or not a shingle belongs to a document
2. the ```minhash``` of a document returns a random permutation of the rows and then the first row number it founds where the value is non 0.
    * therefore row 1 for doc 1
    * and row 0 for doc 2

In [18]:
ex_minhash = pd.DataFrame({'shingleID':[1,2,3,4,5,6],'doc1':[0,1,0,0,1,1],'doc2':[1,1,1,0,1,0]})
ex_minhash

,doc1,doc2,shingleID
0,0,1,1
1,1,1,2
2,0,1,3
3,0,0,4
4,1,1,5
5,1,0,6


But this is for this particular permutation, if we had another one like below then the ```minhash``` would return :
   * row 2 for doc1
   * row 1 for doc2

There are a lot of such permutations. Nevertheless we only care about having the same row for each document which mean we have collision of the ```minhash```. Therefore we have only two rows for which this happens : $\text{shingleID} \in \{2,5\}$. Hence the probability that the two doc have the same ```minhash``` (remmember that rows with two zeros do not count as they are not considered by ```minhash```) is the number of rows where both doc value is 1 divided by the number of rows where they are different : $\frac{2}{5}$

In [19]:
ex_minhash.sample(frac=1)

,doc1,doc2,shingleID
0,0,1,1
1,1,1,2
5,1,0,6
3,0,0,4
2,0,1,3
4,1,1,5


We can chose the length of the fingerprint that is returned by the ```minhash```, as we increase the length we get less variance from the random initialisation of the ```minhasher``` but this also greatly increases the memory usage.

#### The principle of LSH

In [20]:
from lsh import minhash
hasher = minhash.MinHasher(seeds=100, char_ngram=5)
fingerprint0 = hasher.fingerprint('Lorem Ipsum dolor sit amet'.encode('utf8'))
len(fingerprint0)

100

Therefore the hash of a document is composed of a given (```seeds```) number of ```minhashes```. We divide this number of ```minhashes``` into a given number of parts (e.g. 5). Since every single ```minhash``` has the jaccard similarity probability of collision then each of the 5 parts will have this probability as well. These parts represent the **locality** in the term LSH.

Then we hash the content of each part using a different hash function to obtain the *binID* that represents the **hashing** part of the method. Into each bin with *binID* we store the entire fingerprint of the document.

The idea is then that we compare documents that fall in the same bins : we will compare their fingerprint which is equivalent to looking at their Jaccard similarity between shingle sets. Since not all documents will fall into the same bins we have reduced the number of potential candidates.

Indeed we call :
* ```seeds``` :  number of ```minihashes``` that compose the fingerprint
* ```bands``` : the number of bins that we want to use

#### Finding candidate duplicates

In [21]:
book_desc_titles.head()

,title,description,imUrl
asin,,,
0001055178,Master Georgie,Beryl Bainbridge seems drawn to disaster. Firs...,http://ecx.images-amazon.com/images/I/51ZSC6TK...
000171287X,The Berenstains' B Book (Bright & Early Books),"By Stan Berenstain and Jan Berenstain, Illustr...",http://ecx.images-amazon.com/images/I/31hjUxZu...
000100039X,The Prophet,"In a distant, timeless place, a mysterious pro...",http://ecx.images-amazon.com/images/I/81ZKLPiv...
0001473905,Rightly Dividing the Word,--This text refers to thePaperbackedition.,http://ecx.images-amazon.com/images/I/61KPC59B...
0001714538,The Berenstain Bears on the Moon (Bright and E...,PreSchool-Grade 2 A delightful tale told in rh...,http://ecx.images-amazon.com/images/I/A1zj84uC...


In [22]:
dump_path = DUMP_FOLDER
get_candidate = candidate_duplicates(book_desc_titles,dump_path,['title'])

Retrieving from : ../../Project-Data/dump/candidate_dup_title_2_100_5_4
Found 6194 items with possible duplicates.


In [23]:
# Let's quickly oberve the type of data that we obtain
element_to_inspect = 67
similarity_keys = list(get_candidate.keys())
print(get_titles(similarity_keys[element_to_inspect],get_candidate,book_desc_titles))
print(get_desc(similarity_keys[element_to_inspect],get_candidate,book_desc_titles))
display_images(similarity_keys[element_to_inspect],get_candidate,book_desc_titles)

['Shock Totem 4: Curious Tales of the Macabre and Twisted', 'Shock Totem 2: Curious Tales of the Macabre and Twisted', 'Shock Totem 3: Curious Tales of the Macabre and Twisted', 'Shock Totem 1: Curious Tales of the Macabre and Twisted']
['"There is a fine array of stories, but what is perhaps most striking is the high quality of the non-fiction pieces.  The interviews, essays, and  columns, are extremely well done.  They are mini works of art in themselves." --Sheila Merritt,Hellnotes', '"Don\'t write offShock Totemas just another \'new fiend on the block\', \'cause this is one horror \'zine that\'s the real deal, without a doubt! Stories that disturb, nonfiction that entices the reader to dig deeper into the topic at hand...count me in as a big fan.Shock Totemrocks." --James Newman, author ofAnimosityandMidnight Rain', '"Fans of horror stories, pounce uponShock Totemmagazine. Fine new stuff!" --Jack Ketchum, author ofOffspring,Peaceable KingdomandThe Woman(withLucky McKee)', '"With co

We can see that the books can greatly vary eventhough they have the same title. Also it seems hard to extract any meaningful data from their description therefore we need to use the Amazon API to try to get the data we are looking for.

### Refining our candidates

#### Using Amazon API to obtain more data about our products
Because of the heterogeneous nature of the data in description we use Amazon API to obtain consistent informations like the name of authors, ISBN etc...

We use a [wrapper](https://pypi.python.org/pypi/python-amazon-simple-product-api) of the amazon API and the documentation can be found [here](http://pythonhosted.org/python-amazon-simple-product-api/).

Also note that given the ASIN of a product one can access the page of the product using one of the two following addresses:
* ```https://www.amazon.co.uk/dp/<ASIN>/```
* ```https://www.amazon.com/dp/<ASIN>/```

In [24]:
book_desc_titles.head()

,title,description,imUrl
asin,,,
0001055178,Master Georgie,Beryl Bainbridge seems drawn to disaster. Firs...,http://ecx.images-amazon.com/images/I/51ZSC6TK...
000171287X,The Berenstains' B Book (Bright & Early Books),"By Stan Berenstain and Jan Berenstain, Illustr...",http://ecx.images-amazon.com/images/I/31hjUxZu...
000100039X,The Prophet,"In a distant, timeless place, a mysterious pro...",http://ecx.images-amazon.com/images/I/81ZKLPiv...
0001473905,Rightly Dividing the Word,--This text refers to thePaperbackedition.,http://ecx.images-amazon.com/images/I/61KPC59B...
0001714538,The Berenstain Bears on the Moon (Bright and E...,PreSchool-Grade 2 A delightful tale told in rh...,http://ecx.images-amazon.com/images/I/A1zj84uC...


In [29]:
amazon_access_file_path = DUMP_FOLDER + "amazon_access"
book_only_candidates,failed = fill_in_with_details(book_desc_titles,amazon_access_file_path,get_candidate,DUMP_FOLDER)

Retrieving from : 
	../../Project-Data/dump/book_only_candidates_with_details
	../../Project-Data/dump/api_failed 
It took 00:00:00.055 to get the data.


In [30]:
len(book_only_candidates[book_only_candidates['authors'].notnull()].index)

10761

We will therefore look at the author in the API of amazon. We will match books depending on if the authors are identical. Because we do not want to crawl the API too many times we will do it for all the items that are the dictionnary ```get_candidate``` (either as key or in the value list). One must be careful because if some ASIN couldn't be resolved then they won't provide any information in the dataframe.

#### Refining the candidate similars based on authors
We will now use the details we obtain from the API to refine the candidate similars.

In [31]:
book_only_candidates.head(5)

,title,description,imUrl,authors,isbn,eisbn,brand,edition,publisher
asin,,,,,,,,,
000100039X,The Prophet,"In a distant, timeless place, a mysterious pro...",http://ecx.images-amazon.com/images/I/81ZKLPiv...,[Tester Gregg],000100039X,None,None,None,Rupa & Co
0001473123,The Book of Revelation,"American Baptist pastor, Bible teacher, and wr...",http://ecx.images-amazon.com/images/I/317G8R2T...,[Clarence Larkin],0001473123,None,None,None,Rev Clarence Larkin Estate
0002222590,The Golden Gate,'Startlingly good...tense...ingenious' Sunday ...,http://ecx.images-amazon.com/images/I/51RjrfZe...,[Alistair MacLean],0002222590,None,Brand: HarperCollins Publishers Ltd,First Edition,HarperCollins Publishers Ltd
0006176909,House of Cards,"'This blood-and-thunder tale, lifelike and tho...",http://ecx.images-amazon.com/images/I/71IaEyKy...,[Michael Dobbs],0006176909,None,Brand: Fontana Collins,New Ed,HarperCollins Publishers Ltd
0006175015,Bloodline,'Absorbing and eminently well-crafted.' New Yo...,http://ecx.images-amazon.com/images/I/41rLB80O...,[Sidney Sheldon],0006175015,None,Brand: HarperCollins,New Ed,HarperCollins Publishers Ltd


We saw examples were the authors list between two books were very similar but names were written differently, therefore we need to check that some typos are not in the names of authors : (e.g. ```{'Ian MacNeill', 'SportMed BC'}{'Ian MacNeil'}``` where MacNeil is written with two 'l' in the first book but only one in the second one). Therefore we will first clean the names (remove accents, nomalize it to lower case) and then try to match then using the Levenstein distance.

In [32]:
book_only_candidates['authors'] = book_only_candidates['authors'].apply(clean_name_in_dataframe)
similars = get_similar_authors(book_only_candidates,get_candidate,0.35)
print("We have found {} bins of candidate similars".format(len(similars.keys())))

We have found 1366 bins of candidate similars


#### Refining the candidate similars further using the titles

Now we might still have problems with books that have highly similar titles and authors but aren't identical. This could be explained by :
* Different Tomes : same title but different number
* educational books that have identical titles except for what they teach
* simply typos or very small differences 
* many other examples 

Example of books that are not similar:
```
Adobe Dreamweaver CS3 Classroom in a Book['adobe creative team']
Adobe Dreamweaver CS4 Classroom in a Book['adobe creative team']
Adobe Dreamweaver CS5 Classroom in a Book['adobe creative team']
Adobe Dreamweaver CS6 Classroom in a Book['adobe creative team']
----------------------------------------------------------------
Promethea, Book 3['alan moore']
Promethea, Book 5['alan moore']
Promethea, Book 1['alan moore', 'j. h. williams', 'mick gray']

```

Example of books that should be considered similar:
``` 
'Light Science and Magic: An Introduction to Photographic Lighting'['fil hunter', 'paul fuqua', 'steven biver']
'Light: Science and Magic: An Introduction to Photographic Lighting'['fil hunter', 'steven biver', 'paul fuqua']
```

We will therefore also need to see if the title are similar enough. We want to avoid false positive as much as possible. We are sure that we could go even further with matching names but prefer to be conservative here. Therefore we only try to normalize as much as possible the titles and see if then the titles taken as sets have an empty symmetric difference.

In [35]:
book_only_candidates['title'] = book_only_candidates['title'].apply(clean_title)
print(clean_title("Promethea, 'Book 3' (whatever junk)"))
very_similars = get_similar_titles(book_only_candidates,similars)

promethea book 3 whatever junk


In [36]:
print("We have found {} bins of very similar books.".format(len(very_similars.keys())))
very_similar_ASIN = list(very_similars.keys())
very_similar_ASIN += [item for sublist in list(very_similars.values()) for item in sublist]
print("We have {} asins for products that have at least one similar product.".format(len(very_similar_ASIN)))

We have found 786 bins of very similar books.
We have 1611 asins for products that have at least one similar product.


# Testing
*Here I put all my work in progress that didn't show any results yet but could be useful later on.*